In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/competitions/titanic/train.csv
/kaggle/input/competitions/titanic/test.csv
/kaggle/input/competitions/titanic/gender_submission.csv


# ****Import Data****

In [2]:
train_data = pd.read_csv("/kaggle/input/competitions/titanic/train.csv")
test_data = pd.read_csv("/kaggle/input/competitions/titanic/test.csv")
train_data.info()
train_data.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [3]:
train_data['Age'] = train_data['Age'].fillna(train_data['Age'].median())

# 2. Define your age "buckets" (bins) and what to name them
# 0 to 2, 2 to 12, 12 to 18, 18 to 65, and 65 to 100
age_bins = [0, 2, 18, 65, 100]
category_names = ['Infant', 'Child', 'Adult', 'Senior']

train_data['Detailed_Age_Group'] = pd.cut(train_data['Age'], bins=age_bins, labels=category_names)
test_data['Detailed_Age_Group'] = pd.cut(test_data['Age'], bins=age_bins, labels=category_names)

In [4]:
# 1. Calculate Family Size
train_data['FamilySize'] = train_data['SibSp'] + train_data['Parch'] + 1
test_data['FamilySize'] = test_data['SibSp'] + test_data['Parch'] + 1

# 2. The Improved Bins!
# 0 to 1 = Single
# 1 to 4 = Small (Highest Survival)
# 4 to 6 = Medium
# 6 to 15 = Large (Lowest Survival)
size_bins = [0, 1, 4, 6, 15]
fam_category_names = ['Single', 'SmallFam', 'MidFam', 'LargeFam']

# 3. Cut the data
train_data['FamCat'] = pd.cut(train_data['FamilySize'], bins=size_bins, labels=fam_category_names)
test_data['FamCat'] = pd.cut(test_data['FamilySize'], bins=size_bins, labels=fam_category_names)

# Let's see the survival rates for these new groups!
print(train_data[['FamCat', 'Survived']].groupby('FamCat').mean())

          Survived
FamCat            
Single    0.303538
SmallFam  0.578767
MidFam    0.162162
LargeFam  0.160000


/tmp/ipykernel_17/440917484.py:18: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(train_data[['FamCat', 'Survived']].groupby('FamCat').mean())


In [5]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

numerical_transformer = SimpleImputer(strategy='median')

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore')) 
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, ['Fare']), 
        ('cat', categorical_transformer, ['Sex', 'Pclass', 'Embarked', 'Detailed_Age_Group', 'FamCat'])
    ])

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression

# 1. Define the 3 voters (We leave the parameters blank so GridSearch can fill them!)
model_1 = RandomForestClassifier(random_state=42)
model_2 = XGBClassifier(random_state=42, eval_metric='logloss')
model_3 = LogisticRegression(max_iter=1000, random_state=42)

# 2. Create the Committee
ensemble_model = VotingClassifier(
    estimators=[
        ('rf', model_1), 
        ('xgb', model_2), 
        ('log_reg', model_3)
    ], 
    voting='hard' 
)

# 3. Put the Committee in your Pipeline
# (Assuming you still have your 'preprocessor' from earlier)
my_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor), 
    ('committee', ensemble_model)   
])

# 4. Set up the Grid Search Parameters using the "Double Underscore" rule!
param_grid = {
    # Tune the Random Forest
    'committee__rf__n_estimators': [100, 300],
    'committee__rf__max_depth': [5, 7],
    
    # Tune the XGBoost
    'committee__xgb__learning_rate': [0.05, 0.1],
    'committee__xgb__max_depth': [3, 5],
    
    # Tune the Logistic Regression ('C' is how strictly it draws its lines)
    'committee__log_reg__C': [0.1, 1.0]
}

# 5. Run the Search!
print("Starting the massive Ensemble Grid Search...")
# cv=5 means 5-fold cross-validation
grid_search = GridSearchCV(my_pipeline, param_grid, cv=5, scoring="accuracy", n_jobs=-1)

features = ['Fare', 'Sex', 'Pclass', 'Embarked', 'Detailed_Age_Group', 'FamCat']

X = train_data[features]
y = train_data["Survived"]
X_test = test_data[features]

grid_search.fit(X, y)

print(f"Best Ensemble Score: {grid_search.best_score_:.4f}")
print("Best Parameters:", grid_search.best_params_)

Starting the massive Ensemble Grid Search...
Best Ensemble Score: 0.8182
Best Parameters: {'committee__log_reg__C': 0.1, 'committee__rf__max_depth': 7, 'committee__rf__n_estimators': 100, 'committee__xgb__learning_rate': 0.05, 'committee__xgb__max_depth': 5}


In [7]:
from sklearn.model_selection import cross_val_score

# Assuming you have already defined your 'my_pipeline', 'X', and 'y'
print("Running Cross-Validation...")

# cv=5 means 5-fold cross-validation
# n_jobs=-1 makes it run as fast as possible
scores = cross_val_score(my_pipeline, X, y, cv=5, scoring='accuracy', n_jobs=-1)

# Print the results
print("Scores for each of the 5 folds:", scores)
print(f"Average CV Score (Expected Kaggle Score): {scores.mean():.4f}")

Running Cross-Validation...
Scores for each of the 5 folds: [0.81005587 0.80898876 0.85393258 0.80337079 0.84269663]
Average CV Score (Expected Kaggle Score): 0.8238


In [8]:
predictions = grid_search.predict(X_test)
output = pd.DataFrame({"PassengerId": test_data["PassengerId"], "Survived": predictions})
output.to_csv("submission.csv", index=False)
print("Saved Ensemble submission file!")

Saved Ensemble submission file!
